# Using commuting projectors in STRUPHY

In STRUPHY the commuting projectors of the de Rham sequence are based on __inter-/histopolation__ of spline basis functions at Greville points. 

There are two kinds of projectors in STRUPHY:

* __global projectors__ based on inter-/histopolation in the __whole domain__ 
* __local projectors__ based on quasi-inter-/histopolation in a __neighbourhood of $x_i$__

In [ ]:
import numpy             as np
import matplotlib.pyplot as plt
import time
from sys import getsizeof

import hylife.utilitis_FEEC.projectors_global as proj_glob
import hylife.utilitis_FEEC.projectors_global_fast as proj_glob_fast
import hylife.utilitis_FEEC.projectors_global_fastV2 as proj_glob_fastV2
import hylife.utilitis_FEEC.projectors_global_fastV3 as proj_glob_fastV3

import hylife.utilitis_FEEC.evaluationV2 as evaV2
from hylife.utilitis_FEEC.linalg_kron import kron_matvec_3d


import hylife.utilitis_FEEC.bsplines as bsp

## Global projectors in 3D 

In [ ]:
# ------------------------
# which projector to test (0, 11, 12, 13, 21, 22, 23, 3):
# ------------------------
comp = 3

# ------------------------
# function to be projected
# ------------------------
fun = lambda xi1, xi2, xi3 : np.sin( np.pi*( np.sin(2.*np.pi*xi1)
                                            *np.sin(2.*np.pi*xi2)
                                            *np.sin(2.*np.pi*xi3) ) )

#-----------------
# Create the grid:
#-----------------
# side lengths of logical cube
L = [3., 2., 1.] 

# spline degrees
p = [3, 3, 3]   

# periodic boundary conditions (use 'False' if clamped)
bc = [True, True, True] 

# errors:
before_max = 0.
before_L2  = 0.
before_L1  = 0.
before_max_fast = 0.
before_L2_fast  = 0.
before_L1_fast  = 0.
before_max_fastV2 = 0.
before_L2_fastV2  = 0.
before_L1_fastV2  = 0.
before_max_fastV3 = 0.
before_L2_fastV3  = 0.
before_L1_fastV3  = 0.

# loop over different number of elements (convergence test)
Nel_cases = [2**n for n in range(2,4)]

# loop over different number of quadrature points per element
Nq_cases = [6]

for Nel_k in Nel_cases:
    
    
    # number of elements
    Nel = [3*Nel_k, 2*Nel_k, Nel_k]   
    print('Nel=', Nel)

    # element boundaries
    el_b = [np.linspace(0., L_i, Nel_i + 1) for L_i, Nel_i in zip(L, Nel)] 

    # knot sequences
    T = [bsp.make_knots(el_b_i, p_i, bc_i) for el_b_i, p_i, bc_i in zip(el_b, p, bc)] 
    
    for Nq_k in Nq_cases:
        
        # number of quadrature points
        Nq = [Nq_k, Nq_k, Nq_k]

        # create an instance of the projector class
        obj      = proj_glob.projectors_3d(T, p, bc, Nq)
        obj_fast = proj_glob_fast.projectors_3d(T, p, bc, Nq)
        obj_fastV2 = proj_glob_fastV2.projectors_3d(T, p, bc, Nq)
        obj_fastV3 = proj_glob_fastV3.projectors_3d(T, p, bc, Nq)
        # ---------------------------------------------
        # create LU decomposition of necessary matrices
        # ---------------------------------------------
        if comp==0:
            obj.NNN_LU()   # for PI_0
            obj_fast.NNN_LU()   
            obj_fastV2.NNN_LU()   
        elif comp==11:
            obj.DNN_LU()  # for PI_11
            obj_fast.DNN_LU()  
            obj_fastV2.DNN_LU()  
        elif comp==12:
            obj.NDN_LU()  # for PI_12
            obj_fast.NDN_LU()  
            obj_fastV2.NDN_LU()  
        elif comp==13:
            obj.NND_LU()  # for PI_13
            obj_fast.NND_LU()  
            obj_fastV2.NND_LU()  
        elif comp==21:
            obj.NDD_LU()  # for PI_21
            obj_fast.NDD_LU() 
            obj_fastV2.NDD_LU() 
        elif comp==22:
            obj.DND_LU()  # for PI_22
            obj_fast.DND_LU()  
            obj_fastV2.DND_LU() 
        elif comp==23:
            obj.DDN_LU()  # for PI_23
            obj_fast.DDN_LU()  
            obj_fastV2.DDN_LU()  
        elif comp==3:
            obj.DDD_LU()  # for PI_3
            obj_fast.DDD_LU()  
            obj_fastV2.DDD_LU()  

        # --------------------
        # compute coefficients
        # --------------------
        t0 = time.time()
        
        if comp==0:
            coeffs = obj.PI_0(fun)
        elif comp==11:
            coeffs = obj.PI_11(fun)
        elif comp==12:
            coeffs = obj.PI_12(fun)
        elif comp==13:
            coeffs = obj.PI_13(fun)
        elif comp==21:
            coeffs = obj.PI_21(fun)
        elif comp==22:
            coeffs = obj.PI_22(fun)
        elif comp==23:
            coeffs = obj.PI_23(fun)
        elif comp==3:
            coeffs = obj.PI_3(fun)
            
        t1 = time.time()
        print('time, original  =     %10.7e' % (t1-t0))
        
        t0 = time.time()

        if comp==0:
            mat_f = obj_fast.eval_for_PI_0(fun) 
        elif comp==11:
            mat_f = obj_fast.eval_for_PI_11(fun) 
        elif comp==12:
            mat_f = obj_fast.eval_for_PI_12(fun) 
        elif comp==13:
            mat_f = obj_fast.eval_for_PI_13(fun) 
        elif comp==21:
            mat_f = obj_fast.eval_for_PI_21(fun) 
        elif comp==22:
            mat_f = obj_fast.eval_for_PI_22(fun) 
        elif comp==23:
            mat_f = obj_fast.eval_for_PI_23(fun) 
        elif comp==3:
            mat_f = obj_fast.eval_for_PI_3(fun) 

        t1 = time.time()
        dt = t1-t0

        t0 = time.time()

        if comp==0:
            coeffs_fast = obj_fast.PI_0_mat( mat_f )
        elif comp==11:
            coeffs_fast = obj_fast.PI_11_mat( mat_f  )
        elif comp==12:
            coeffs_fast = obj_fast.PI_12_mat( mat_f  )
        elif comp==13:
            coeffs_fast = obj_fast.PI_13_mat( mat_f  )
        elif comp==21:
            coeffs_fast = obj_fast.PI_21_mat( mat_f  )
        elif comp==22:
            coeffs_fast = obj_fast.PI_22_mat( mat_f  )
        elif comp==23:
            coeffs_fast = obj_fast.PI_23_mat( mat_f  )
        elif comp==3:
            coeffs_fast = obj_fast.PI_3_mat( mat_f  )

        t1 = time.time() 

        print('time_fast  , eval + integrate & project, %10.7e + %10.7e = %10.7e' % (dt,(t1-t0),dt+(t1-t0)))
        t0 = time.time()
        
        mat_f = obj_fastV2.eval_for_PI(comp,fun)
        

        t1 = time.time()
        dt = t1-t0

        t0 = time.time()
        
        if comp==0:
            coeffs_fastV2 = obj_fastV2.PI_0_mat( mat_f )
        elif comp==11:
            coeffs_fastV2 = obj_fastV2.PI_11_mat( mat_f  )
        elif comp==12:
            coeffs_fastV2 = obj_fastV2.PI_12_mat( mat_f  )
        elif comp==13:
            coeffs_fastV2 = obj_fastV2.PI_13_mat( mat_f  )
        elif comp==21:
            coeffs_fastV2 = obj_fastV2.PI_21_mat( mat_f  )
        elif comp==22:
            coeffs_fastV2 = obj_fastV2.PI_22_mat( mat_f  )
        elif comp==23:
            coeffs_fastV2 = obj_fastV2.PI_23_mat( mat_f  )
        elif comp==3:
            coeffs_fastV2 = obj_fastV2.PI_3_mat( mat_f  )

        t1 = time.time()
        print('time_fastV2, eval + integrate & project, %10.7e + %10.7e = %10.7e' % (dt,(t1-t0),dt+(t1-t0)))
        t0 = time.time()
        
        mat_f = obj_fastV3.eval_for_PI(comp,fun)
        

        t1 = time.time()
        dt = t1-t0
        t0 = time.time()
        
        coeffs_fastV3 = obj_fastV3.PI_mat(comp, mat_f )

        t1 = time.time()
        print('time_fastV3, eval + integrate & project, %10.7e + %10.7e = %10.7e' % (dt,(t1-t0),dt+(t1-t0)))
        print('post processing...')
        
        #print('size of mat_f in MB:', getsizeof( obj_fast.eval_for_PI_21(fun) )*1e-6)

        # grid points for error computation
        pts_loc, wts_loc = np.polynomial.legendre.leggauss(5)   # quadrature points per element

        pts1, wts1 = bsp.quadrature_grid(el_b[0], pts_loc, wts_loc)   
        pts2, wts2 = bsp.quadrature_grid(el_b[1], pts_loc, wts_loc) 
        pts3, wts3 = bsp.quadrature_grid(el_b[2], pts_loc, wts_loc) 

        xgrid = [ pts1.flatten(), pts2.flatten(), pts3.flatten() ]   # error grid
        
        # projection evaluated at the grid points

        t0 = time.time()
        f_basemat= evaV2.FEM_evalbase_3d(comp, xgrid, T, p, bc)
        t1 = time.time()
        print('time, eval FEM base V2  =     %10.7e' % (t1-t0))
        t0 = time.time()
        f_h      = kron_matvec_3d(f_basemat,coeffs)
        f_h_fast = kron_matvec_3d(f_basemat,coeffs_fast)
        f_h_fastV2 = kron_matvec_3d(f_basemat,coeffs_fastV2)
        f_h_fastV3 = kron_matvec_3d(f_basemat,coeffs_fastV3)
        t1 = time.time()
        print('time, eval 4xFEM fields V2=     %10.7e' % (t1-t0))
        t0 = time.time()
        
        # exact function evaluated at the grid points
        f = np.empty((xgrid[0].size, xgrid[1].size, xgrid[2].size))

        pts_1,pts_2,pts_3=np.meshgrid(xgrid[0],xgrid[1],xgrid[2],indexing='ij',sparse=True)
        f = fun(pts_1,pts_2,pts_3)
        
        t1 = time.time()
        print('time, eval exact func  =     %10.7e' % (t1-t0))
        # max-norm
        err_max      = np.max( np.abs(f - f_h) )
        err_max_fast = np.max( np.abs(f - f_h_fast) )
        err_max_fastV2 = np.max( np.abs(f - f_h_fastV2) )
        err_max_fastV3 = np.max( np.abs(f - f_h_fastV3) )
        
        # L2-norm
        err_L2      = np.sqrt( np.sum( np.abs(f - f_h)**2 ) / np.size(f) )
        err_L2_fast = np.sqrt( np.sum( np.abs(f - f_h_fast)**2 ) / np.size(f) )
        err_L2_fastV2 = np.sqrt( np.sum( np.abs(f - f_h_fastV2)**2 ) / np.size(f) )
        err_L2_fastV3 = np.sqrt( np.sum( np.abs(f - f_h_fastV3)**2 ) / np.size(f) )

        # L1-norm
        err_L1      = np.sum( np.abs(f - f_h) ) / np.size(f) 
        err_L1_fast = np.sum( np.abs(f - f_h_fast) ) / np.size(f) 
        err_L1_fastV2 = np.sum( np.abs(f - f_h_fastV2) ) / np.size(f) 
        err_L1_fastV3 = np.sum( np.abs(f - f_h_fastV3) ) / np.size(f)

        # print errors
        print( 'orig,  Nq=', Nq,
               '{:10.7f}'.format(err_max), '{:10.7f}'.format(before_max/err_max), ' | ',
               '{:10.7f}'.format(err_L2),  '{:10.7f}'.format(before_L2/err_L2),   ' | ', 
               '{:10.7f}'.format(err_L1),  '{:10.7f}'.format(before_L1/err_L1),   ' | ')
        
        print( 'fast,  Nq=', Nq,
               '{:10.7f}'.format(err_max_fast), '{:10.7f}'.format(before_max_fast/err_max_fast), ' | ',
               '{:10.7f}'.format(err_L2_fast),  '{:10.7f}'.format(before_L2_fast/err_L2_fast),   ' | ', 
               '{:10.7f}'.format(err_L1_fast),  '{:10.7f}'.format(before_L1_fast/err_L1_fast),   ' | ')
        
        print( 'fastV2,Nq=', Nq,
               '{:10.7f}'.format(err_max_fastV2), '{:10.7f}'.format(before_max_fastV2/err_max_fastV2), ' | ',
               '{:10.7f}'.format(err_L2_fastV2),  '{:10.7f}'.format(before_L2_fastV2/err_L2_fastV2),   ' | ', 
               '{:10.7f}'.format(err_L1_fastV2),  '{:10.7f}'.format(before_L1_fastV2/err_L1_fastV2),   ' | ')
        print( 'fastV3,Nq=', Nq,
               '{:10.7f}'.format(err_max_fastV3), '{:10.7f}'.format(before_max_fastV3/err_max_fastV3), ' | ',
               '{:10.7f}'.format(err_L2_fastV3),  '{:10.7f}'.format(before_L2_fastV3/err_L2_fastV3),   ' | ', 
               '{:10.7f}'.format(err_L1_fastV3),  '{:10.7f}'.format(before_L1_fastV3/err_L1_fastV3),   ' | ')

        before_max = err_max
        before_L2  = err_L2
        before_L1  = err_L1
        
        before_max_fast = err_max_fast
        before_L2_fast  = err_L2_fast
        before_L1_fast  = err_L1_fast
        
        before_max_fastV2 = err_max_fastV2
        before_L2_fastV2  = err_L2_fastV2
        before_L1_fastV2  = err_L1_fastV2
        
        before_max_fastV3 = err_max_fastV3
        before_L2_fastV3  = err_L2_fastV3
        before_L1_fastV3  = err_L1_fastV3        
        print('done.')
        
    print('')